In [1]:
println(s"Current spark version is ${spark.version}")

Current spark version is 2.4.0


In [2]:
import org.apache.spark.sql.types.{StructType, StructField, IntegerType, LongType, StringType}

val dataSchema = new StructType()
    .add("target", IntegerType)
    .add("id", LongType)
    .add("raw_timestamp", StringType)
    .add("query_status", StringType)
    .add("author", StringType)
    .add("tweet", StringType)

    
val dataPath= "/home/jovyan/data/training.1600000.processed.noemoticon.csv"

val raw_sentiment = spark.read
    .format("csv")
    .option("header",false)
    .schema(dataSchema)
    .load(dataPath)
    .selectExpr("(case when target=4 then 1 else 0 end) as label","tweet")

raw_sentiment.groupBy($"label").count.show

+-----+------+
|label| count|
+-----+------+
|    1|800000|
|    0|800000|
+-----+------+



dataSchema = StructType(StructField(target,IntegerType,true), StructField(id,LongType,true), StructField(raw_timestamp,StringType,true), StructField(query_status,StringType,true), StructField(author,StringType,true), StructField(tweet,StringType,true))
dataPath = /home/jovyan/data/training.1600000.processed.noemoticon.csv
raw_sentiment = [label: int, tweet: string]


[label: int, tweet: string]

In [3]:
import org.apache.spark.ml.{Pipeline, PipelineModel}
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.{HashingTF, Tokenizer}
import org.apache.spark.ml.linalg.Vector
import org.apache.spark.sql.Row

val tokenizer = new Tokenizer()
    .setInputCol("tweet")
    .setOutputCol("words")

val hashingTF = new HashingTF()
    .setNumFeatures(1000)
    .setInputCol(tokenizer.getOutputCol)
    .setOutputCol("features")

val lr = new LogisticRegression()
    .setMaxIter(10)
    .setRegParam(0.001)

val pipeline = new Pipeline()
  .setStages(Array(tokenizer, hashingTF, lr))


tokenizer = tok_45185f5bacef
hashingTF = hashingTF_75fd8de54d7e
lr = logreg_0d00d2fc50fb
pipeline = pipeline_5dbd3ec15599


pipeline_5dbd3ec15599

In [4]:
val model = pipeline.fit(raw_sentiment)

model = pipeline_5dbd3ec15599


pipeline_5dbd3ec15599

In [5]:
model.write.overwrite().save("/home/jovyan/models/spark-ml-model")

In [6]:
val sameModel = PipelineModel.load("/home/jovyan/models/spark-ml-model")

sameModel = pipeline_5dbd3ec15599


pipeline_5dbd3ec15599

In [7]:
val predictionsDF = sameModel.transform(raw_sentiment)

predictionsDF.show()

+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|label|               tweet|               words|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|    0|@switchfoot http:...|[@switchfoot, htt...|(1000,[7,14,21,54...|[-0.9010125659409...|[0.28884245921770...|       1.0|
|    0|is upset that he ...|[is, upset, that,...|(1000,[170,193,22...|[1.84195706807782...|[0.86318000204746...|       0.0|
|    0|@Kenichan I dived...|[@kenichan, i, di...|(1000,[10,36,77,1...|[1.56488554961195...|[0.82705328017353...|       0.0|
|    0|my whole body fee...|[my, whole, body,...|(1000,[82,191,296...|[0.22286270195736...|[0.55548620895380...|       0.0|
|    0|@nationwideclass ...|[@nationwideclass...|(1000,[18,96,130,...|[3.23587893775244...|[0.96216236372478...|       0.0|
|    0|@

predictionsDF = [label: int, tweet: string ... 5 more fields]


[label: int, tweet: string ... 5 more fields]

In [8]:
import org.apache.spark.sql.functions._

val getProbability = udf((prediction: org.apache.spark.ml.linalg.Vector) => prediction(1))

getProbability = UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))


UserDefinedFunction(<function1>,DoubleType,Some(List(org.apache.spark.ml.linalg.VectorUDT@3bfc3ba7)))

In [1]:
predictionsDF
.select($"label"
        ,getProbability($"probability").alias("clean_probability")
        ,$"tweet"
       
       )
.orderBy($"clean_probability".desc)
.limit(100)
.show(false)


+-----+------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+
|label|clean_probability |tweet                                                                                                                                                   |
+-----+------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------+
|1    |0.9999999976120344|? ? ? ? ? ? ? ?? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?Take that                 |
|1    |0.9999999839831921|? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?    ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?     ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ? ?  twitterart                    |
|0    |0.9999999614621983|@Werecat1  It cut off my thank you! - thank you thank you thank you thank 